### Demo na konferencję EuroHPC (case z kodowaniem binarnym i z QAOA) - draft
DO ZMIANY: opis 
We will focus on the following workflow instance (for now, in the simplified problem, we will not use the Workflow or WorkflowSchedulingProblem classes, because we want to test the Unbalanced Penalization method https://arxiv.org/pdf/2211.13914.pdf), which is not included in QHyper. But I will leave it here just for reference (especially for observation of what data is in the time and cost matrices).

In [1]:
import sys
sys.path.append("..")
from QHyper.problems.workflow_scheduling import (
    Workflow,
    WorkflowSchedulingBinary
)

tasks_file =  "workflows_data/workflows/3_tasks_4_machines_1_path.json"
machines_file = "workflows_data/machines/machines_for_3_tasks_4_machines_1_path.json"
deadline = 13

workflow = Workflow(tasks_file, machines_file, deadline)
problem = WorkflowSchedulingBinary(workflow)


In [2]:
problem.workflow.cost_matrix


,MachineA,MachineB,MachineC,MachineD
task1,6.0,8.0,8.0,2.0
task2,3.0,4.0,4.0,1.0
task3,12.0,16.0,16.0,4.0


Uwaga: tutaj tak naprawdę jest dodana jeszcze jedna maszyna (liczba maszyn musi być 2^n)

In [3]:
problem.workflow.time_matrix


,MachineA,MachineB,MachineC,MachineD
task1,6.0,2.0,4.0,16.0
task2,3.0,1.0,2.0,8.0
task3,12.0,4.0,8.0,32.0


Below is the list of the hyperparameters (weights) that can be changed.  I assumed that the weights hyper_b, hyper_c, hyper_d can be the same, since the constraint they represent has a very similar form.

In [4]:
hyper_params = {'cost_function_weight': 1, # weight for: cost function
               'deadline_linear_form_weight': 2, # weight for: deadline constraint - linear form (-- this is from the unbalanced penalization approach)
                'deadline_quadratic_form_weight': 2} # weight for: deadline constraint - quadratic form


 DO ZMIANY: generacja tych wzorów z WFCommons

In [5]:
print(f"Variables used to describe objective function"
      f" and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints (RHS == 0):")
for constraint in problem.constraints:
    print(f"    {constraint}")


Variables used to describe objective function and constraints: (x0, x1, x2, x3, x4, x5)
Objective function: {('x0', 'x1'): -8.0, ('x0',): 2.0, ('x1',): 2.0, ('x2', 'x3'): -4.0, ('x2',): 1.0, ('x3',): 1.0, ('x4', 'x5'): -16.0, ('x4',): 4.0, ('x5',): 4.0, (): 21.0}
Constraints (RHS == 0):
    {('x0', 'x1'): 16.0, ('x0',): -2.0, ('x1',): -4.0, ('x2', 'x3'): 8.0, ('x2',): -1.0, ('x3',): -2.0, ('x4', 'x5'): 32.0, ('x4',): -4.0, ('x5',): -8.0, (): 21.0} <= 13


In [6]:
from QHyper.solvers.converter import Converter

In [7]:
result = Converter.create_qubo(problem, [1,0.5,2])

In [8]:
# result

# 3 Using QHyper

Do zmiany - konfiguracja wag

In [9]:
import numpy as np
params_config = {
        'angles': [[0.1e-13]*6, [np.pi/2]*6], # QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
        'hyper_args': [hyper_params['cost_function_weight'], 
                       hyper_params['deadline_linear_form_weight'],
                       hyper_params['deadline_quadratic_form_weight']
                       ],
    }

In [10]:
# Create a VQA instance with QAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)
# DO ZMIANY sqaoa -> qaoa (backend= "default.qubit") oraz QNG optimiser (krok 0.0001) 50 krokow
from QHyper.solvers import VQA

solver_config = {
    "pqc": {
        "type": "qml_qaoa",
        "layers": 6,
        "optimizer": "qng",
        "optimizer_args": {
            "stepsize": 0.00045,
            "steps": 50,
            "verbose": True,
        },
        "backend": "default.qubit",
    },
    "params_inits": params_config
}
vqa = VQA.from_config(problem, config=solver_config)


In [11]:
solver_results = vqa.solve()


summand None, type: <class 'NoneType'>
summand   (-1588.0) [Ix0]
+ (1588.0) [Zx0], type: <class 'pennylane.numpy.tensor.tensor'>


AttributeError: 'tensor' object has no attribute 'wires'

In [ ]:
print(f"Best params: {solver_results.params}")


In [ ]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=20, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.results_probabilities, limit_results=20)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=0)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


In [ ]:
import pandas as pd

pd.DataFrame(solver_results.history[0]).reset_index().plot(x='index',y='value', kind='scatter')
